# Optimizing Page Layouts and Advertisements  
# using Reinforcement Learning  
It's pretty hard to test reinforcement learning algorithms without some interaction with the environment, but it's also hard to give all the users of this course access to a live system to collect data on real customers, so the experiments will all be based on simulations. All of the code should work on live systems, however.

# Thompson Sampling
Note that Thompson Sampling can change based on the distribution of rewards. Since a lot of business cases involve users either taking an action or not taking an action (replying to email, subscribing, clicking on ad, buying product, viewing page, etc.), we use a Bernoulli distribution (weighted coin flip) to represent the distribution over outcomes.

In [ ]:
import numpy as np
import math

def mean_std(list_of_numbers):
    return np.mean(list_of_numbers), np.std(list_of_numbers)

def mean_ste(list_of_numbers):
    mean, std = mean_std(list_of_numbers)
    return mean, std/math.sqrt(len(list_of_numbers))

def biased(list_of_numbers):
    #Prevent std from being zero by adding phantom observations.
    new_list = list_of_numbers.copy()
    new_list.extend([1, 0])
    return new_list

class GaussianThompsonSampler:
    """Gaussians describe the statistics around means very well.
        This class implements Thompson Sampling based on this."""
    
    @staticmethod
    def sample_arms(means, std_errs):
        """Takes arm statistics and returns a sampled index.
            std_err is deviation in the mean, called the standard error."""
        # means and std_devs are assumed to correspond element for element.
        samples = means.copy()
        for index in range(len(means)):
            if std_errs[index] > 0:
                samples[index] = np.random.normal(means[index], std_errs[index])
        max_ind = max(range(len(means)), key=lambda index: samples[index])
        return max_ind
    
    @staticmethod
    def sample_from_scores(scores_matrix):
        """scores_matrix is a list of lists of scores."""
        means = []
        std_errs = []
        for scores_list in scores_matrix:
            mean, ste = mean_ste(biased(scores_list))
            means.append(mean)
            std_errs.append(ste)
        return GaussianThompsonSampler.sample_arms(means, std_errs)

In [ ]:
arms = ("a", "b")
scores_a = []
scores_b = []
for i in range(20):
    #print(test_bts.max_sample(arms))
    sampled_arm = arms[GaussianThompsonSampler.sample_from_scores([scores_a, scores_b])]
    print(sampled_arm)
    if sampled_arm == "a":
        scores_a.append(0)
    else:
        scores_b.append(1)

# Infinite-Arm Thompson Sampling
We add in a special arm/action representing drawing a new action from the pool.

In [ ]:
class InfiniteThompsonSampler:
    @staticmethod
    def sample_arms(means, std_errs):
        """Takes arm statistics and returns a sampled index.
            std_err is deviation in the mean, called the standard error."""
        # means and std_devs are assumed to correspond element for element.
        samples = means.copy()
        for index in range(len(means)):
            if std_errs[index] > 0:
                samples[index] = np.random.normal(means[index], std_errs[index])
        # Define the mean and std_dev of value for a new arm
        new_mean, new_std = mean_std(biased(means)) # Prevent deterministic non-sampling
        samples.append(np.random.normal(new_mean, new_std))
        max_ind = max(range(len(samples)), key=lambda index: samples[index])
        return max_ind
    
    @staticmethod
    def sample_from_scores(scores_matrix):
        """scores_matrix is a list of lists of scores."""
        means = []
        std_errs = []
        for scores_list in scores_matrix:
            mean, ste = mean_ste(biased(scores_list))
            means.append(mean)
            std_errs.append(ste)
        return InfiniteThompsonSampler.sample_arms(means, std_errs)
    

In [ ]:
import matplotlib.pyplot as plt

# Test infinite arm-sampling.
max_trials = 1000
means=[np.random.uniform() for arm in range(max_trials)]
print("sample means: ", means[:10])
plt.plot(means, [1 for i in range(max_trials)], ".")
plt.show()

all_cumulative_rewards = []
sampler = InfiniteThompsonSampler
print(sampler)
cumulative_rewards = []
cumulative_reward = 0.0
scores = []
for trial in range(max_trials):
    index = sampler.sample_from_scores(scores)
    if index >= len(scores): # If new arm
        scores.append([])
    reward = np.random.binomial(1, means[index])
    cumulative_reward += reward
    cumulative_rewards.append(cumulative_reward)
    scores[index].append(reward) # That is, add 1 if it was a success, 0 if failure.
all_cumulative_rewards.append(cumulative_rewards)
print("Sampled from: ", len(totals))
plt.plot(range(max_trials), range(max_trials), label="Benefit of Hindsight")
plt.plot(range(max_trials), all_cumulative_rewards[0], label="Thompson")
plt.legend()
plt.show()

# Contextual Bandit Process

We're going to make use of a free package called `GPy`. You can install it with the commands below, or find it on GitHub at https://github.com/SheffieldML/GPy

In [ ]:
# These commands will install gpy on the system. It may take a while.
#!conda update scipy
!pip install gpy

In [ ]:
import math
# Define a simple test - context is randomly chosen between 0 and 1 uniformly.
# Actions are either .1, .2, .3, .4, .5 ... 1.0
actions = [.1 * i for i in range(1,10 + 1)]
def true_reward(context, action):
    return (math.sin(context * 3 + action) + 1)/2

def noisy_reward(context, action):
    return np.random.binomial(1, true_reward(context, action) )

In [ ]:
import GPy
import numpy as np

num_steps = 100
total_rewards = []
total_r = 0
true_rewards = []
total_true_r = 0

sampled_X = []
sampled_Y = []

for step in range(num_steps):
    context = np.random.random() # Random value between 0 and 1
    
    context_actions = np.array([[context] + [action] for action in actions])
    
    # If there are no points, just choose a random action:
    if len(sampled_X) == 0:
        action_index = np.random.choice(range(len(actions)))
    else:
        # Fit a Gaussian process to the points so far and predict rewards for each action.
        kernel = GPy.kern.RBF(input_dim=len(sampled_X[0]))
        model = GPy.models.GPRegression(np.array(sampled_X), np.array(sampled_Y), kernel)
        for i in range(5):
            model.optimize('bfgs', max_iters=100) #Finds good Gaussian Process model parameters
        # Then use a Thompson Sampler
        predicted_means, predicted_stds = model.predict_noiseless(context_actions)
        action_index = GaussianThompsonSampler.sample_arms(predicted_means, predicted_stds)
    
    action = actions[action_index]
    reward = noisy_reward(context, action)
    total_r += reward
    total_rewards.append(total_r)
    
    sampled_X.append(context_actions[action_index])
    sampled_Y.append([reward])
    
    # Compute True Reward for Comparison:
    # This is the value if the reward function were perfectly well known.
    total_true_r += max([true_reward(context, action) for action in actions])
    true_rewards.append(total_true_r)

plt.title("Contextual Bandit vs. Hindsight")
plt.plot(range(num_steps), true_rewards, label="Hindsight")
plt.plot(range(num_steps), total_rewards, label="Contextual Bandit")
plt.legend()
plt.show()